In [1]:
import pyspark.sql.functions as F
from pyspark.sql.functions import concat, col

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1651419156158_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [583]:
df = spark.read.format("csv").option(
    "header", "true").load("s3://vandy-rithik-finalproj/geo_data/geo_downtown.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [584]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id_original: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- emdcardnumber: string (nullable = true)
 |-- time_utc: string (nullable = true)
 |-- time_local: string (nullable = true)
 |-- response_time_sec: string (nullable = true)
 |-- weekend_or_not: string (nullable = true)
 |-- geometry: string (nullable = true)
 |-- incident_id: string (nullable = true)
 |-- dist_to_seg: string (nullable = true)
 |-- xdsegid: string (nullable = true)
 |-- month: string (nullable = true)
 |-- hour_of_day: string (nullable = true)
 |-- day: string (nullable = true)
 |-- window_of_day: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- window_id: string (nullable = true)
 |-- xd_id: string (nullable = true)
 |-- measurement_tstamp: string (nullable = true)
 |-- measurement_tstamp_str: string (nullable = true)
 |-- speed: string (nullable = true)
 |-- average_speed: string (nullable = true)
 |-- reference_sp

In [585]:
# add columns we want for features
df = df.withColumn(
    "speed_difference",
    (F.col('speed').cast('integer') - F.col('reference_speed').cast('integer')))
df = df.select(
    concat(df.month, df.year).alias("month_year"), "precip", "temp", "vis",
    "wind_spd", "id_original", "speed_difference", "month", "year")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [586]:
# get the columns we want to do regression on 
df.createOrReplaceTempView("all_merged")
df2 = spark.sql("SELECT month_year, \
                AVG(speed_difference) AS speed_difference, \
                AVG(precip) AS precipitation, \
                AVG(temp) AS temperature, \
                AVG(vis) AS visibility, \
                AVG(wind_spd) AS wind_speed, \
                AVG(month) AS month, \
                AVG(year) AS year, \
                COUNT(DISTINCT id_original) as num_accidents\
                 FROM all_merged\
                 WHERE month_year != 0\
                 GROUP BY month_year \
                 ORDER BY year, month")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [587]:
df2.show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+--------------------+------------------+------------------+------------------+-----+------+-------------+
|month_year|   speed_difference|       precipitation|       temperature|        visibility|        wind_speed|month|  year|num_accidents|
+----------+-------------------+--------------------+------------------+------------------+------------------+-----+------+-------------+
|     52017|-1.7546296296296295| 0.11919191919191909|22.308989898989903| 15.58181818181818| 3.979090909090908|  5.0|2017.0|           33|
|     62017|-1.9382716049382716| 0.10555555555555547|25.711111111111105|15.531481481481482|2.8466666666666667|  6.0|2017.0|           18|
|     72018|-1.5132850241545894| 0.12536231884057966| 27.50115942028985|14.705797101449274|3.2582608695652175|  7.0|2018.0|           23|
|     82018| -2.168010752688172| 0.14193548387096777|27.423978494623654|15.652688172043009|2.6146236559139786|  8.0|2018.0|           31|
|     92018|  -1.57010582010582|  

In [588]:
print(df2.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

33

In [589]:
feature_columns = df2.columns[1:8]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [590]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [591]:
df_new = assembler.transform(df2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [593]:
df_new.show(10, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+-------------------+------------------+------------------+------------------+-----+------+-------------+-------------------------------------------------------------------------------------------------------------+
|month_year|speed_difference   |precipitation      |temperature       |visibility        |wind_speed        |month|year  |num_accidents|features                                                                                                     |
+----------+-------------------+-------------------+------------------+------------------+------------------+-----+------+-------------+-------------------------------------------------------------------------------------------------------------+
|52017     |-1.7546296296296295|0.11919191919191909|22.308989898989907|15.581818181818182|3.979090909090908 |5.0  |2017.0|33           |[-1.7546296296296295,0.11919191919191909,22.308989898989907,15.581818181818182,3.979090909090908,5.0,2017.0] |
|62017     |

In [594]:
feature_columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['speed_difference', 'precipitation', 'temperature', 'visibility', 'wind_speed', 'month', 'year']

In [595]:
from pyspark.ml.regression import LinearRegression

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [596]:
#split into train, test
from pyspark.sql.functions import asc

train = df_new.withColumn("index", monotonically_increasing_id())
train = train.orderBy(asc("index")).limit(23).drop("index")
train.show(25)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+--------------------+------------------+------------------+------------------+-----+------+-------------+--------------------+
|month_year|   speed_difference|       precipitation|       temperature|        visibility|        wind_speed|month|  year|num_accidents|            features|
+----------+-------------------+--------------------+------------------+------------------+------------------+-----+------+-------------+--------------------+
|     52017|-1.7546296296296295| 0.11919191919191907|22.308989898989903|15.581818181818182|3.9790909090909072|  5.0|2017.0|           33|[-1.7546296296296...|
|     62017|-1.9382716049382716| 0.10555555555555547| 25.71111111111111|15.531481481481483|2.8466666666666667|  6.0|2017.0|           18|[-1.9382716049382...|
|     72018|-1.5132850241545894| 0.12536231884057966| 27.50115942028985|14.705797101449278| 3.258260869565217|  7.0|2018.0|           23|[-1.5132850241545...|
|     82018| -2.168010752688172| 0.14193548387

In [597]:
########## Extract last N rows of the dataframe in pyspark

from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import desc

test = df_new.withColumn("index", monotonically_increasing_id())
test = test.orderBy(desc("index")).limit(10).drop("index")
test.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+--------------------+------------------+------------------+------------------+-----+------+-------------+--------------------+
|month_year|   speed_difference|       precipitation|       temperature|        visibility|        wind_speed|month|  year|num_accidents|            features|
+----------+-------------------+--------------------+------------------+------------------+------------------+-----+------+-------------+--------------------+
|     12021| 1.3090277777777777|  0.2869444444444443| 8.198750000000004|14.931944444444445| 3.890833333333333|  1.0|2021.0|           24|[1.30902777777777...|
|    122020|-1.2664652567975831| 0.41184290030211446| 6.714702920443102|13.861430010070496|3.8351319234642487| 12.0|2020.0|           23|[-1.2664652567975...|
|    112020| 0.7558563248308173|0.007496095783446126|12.066996356064548|15.629186187749434| 2.440635085892765| 11.0|2020.0|           27|[0.75585632483081...|
|    102020| -3.823632130384168| 0.08940628637

In [598]:
algo = LinearRegression(
    featuresCol="features",
    labelCol="num_accidents")  # label column is the predicted variable

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [599]:
model = algo.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [613]:
evaluation_summary = model.evaluate(test)
print(evaluation_summary.rootMeanSquaredError, evaluation_summary.r2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

17.052289173439732 -6.825095964871094

In [601]:
predictions = model.transform(test)
predictions.show(15)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+--------------------+------------------+------------------+------------------+-----+------+-------------+--------------------+------------------+
|month_year|   speed_difference|       precipitation|       temperature|        visibility|        wind_speed|month|  year|num_accidents|            features|        prediction|
+----------+-------------------+--------------------+------------------+------------------+------------------+-----+------+-------------+--------------------+------------------+
|     12021| 1.3090277777777777|  0.2869444444444443| 8.198750000000004|14.931944444444444| 3.890833333333333|  1.0|2021.0|           24|[1.30902777777777...|37.037395290473796|
|    122020|-1.2664652567975831| 0.41184290030211435| 6.714702920443102|13.861430010070494| 3.835131923464249| 12.0|2020.0|           23|[-1.2664652567975...|39.783583681874006|
|    112020| 0.7558563248308173|0.007496095783446127|12.066996356064548|15.629186187749434| 2.440635085892765|

In [602]:
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(featuresCol="features", labelCol="num_accidents")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [614]:
model = rf.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [604]:
predictions = model.transform(test)
predictions.show(15)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+--------------------+------------------+------------------+------------------+-----+------+-------------+--------------------+------------------+
|month_year|   speed_difference|       precipitation|       temperature|        visibility|        wind_speed|month|  year|num_accidents|            features|        prediction|
+----------+-------------------+--------------------+------------------+------------------+------------------+-----+------+-------------+--------------------+------------------+
|     12021| 1.3090277777777777|  0.2869444444444443| 8.198750000000004|14.931944444444442| 3.890833333333333|  1.0|2021.0|           24|[1.30902777777777...|33.480000000000004|
|    122020|-1.2664652567975831|  0.4118429003021144| 6.714702920443103|13.861430010070494| 3.835131923464249| 12.0|2020.0|           23|[-1.2664652567975...|              33.7|
|    112020| 0.7558563248308173|0.007496095783446126|12.066996356064548|15.629186187749434| 2.440635085892765|

In [605]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="num_accidents",
                                predictionCol="prediction",
                                metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 13.8383

In [606]:
from pyspark.ml.regression import GBTRegressor

gbt = GBTRegressor(featuresCol="features", labelCol="num_accidents", maxIter=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [607]:
model = gbt.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [611]:
predictions = model.transform(test)
predictions.show(15)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+--------------------+------------------+------------------+------------------+-----+------+-------------+--------------------+------------------+
|month_year|   speed_difference|       precipitation|       temperature|        visibility|        wind_speed|month|  year|num_accidents|            features|        prediction|
+----------+-------------------+--------------------+------------------+------------------+------------------+-----+------+-------------+--------------------+------------------+
|     12021| 1.3090277777777777|  0.2869444444444443| 8.198750000000004|14.931944444444445| 3.890833333333333|  1.0|2021.0|           24|[1.30902777777777...|29.721716257109332|
|    122020|-1.2664652567975831| 0.41184290030211446| 6.714702920443102|13.861430010070496|3.8351319234642487| 12.0|2020.0|           23|[-1.2664652567975...|30.000264742229334|
|    112020| 0.7558563248308173|0.007496095783446127| 12.06699635606455|15.629186187749433| 2.440635085892765|

In [609]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="num_accidents",
                                predictionCol="prediction",
                                metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 11.6345

In [610]:
# do linear regression and then do random forest and boosted trees. then choose 1 to present. shouldn't be too bad. then get to visualization

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…